In [109]:
class Value:
    def __init__(self, dictionary):
        self.dictionary = dictionary
        self.key = list(dictionary.keys())[0]
        self.dictionary2 = dictionary[self.key]
        self.format = list(self.dictionary2.keys())[0]
        self.value = self.dictionary2[self.format]
        
class Formula:
    def __init__(self, dictionary):
        self.dictionary = dictionary
        self.key = list(dictionary.keys())[0]
        self.value0 = dictionary[self.key]
        self.key2 = list(self.dictionary[self.key])[0]
        
        if self.key2 == "reference":
            self.formula = '+' + self.value0[self.key2]
        
        if self.key2 == 'sum':
            formula = ''
            for s in self.value0[self.key2]:
                formula += '+'
                testas = Formula(s)
                formula += str(testas.value0)
            self.formula = formula
            
        if self.key2 == 'multiply':
            formula = '='
            for s in self.value0[self.key2]:
                testas = Formula(s)
                formula += str(testas.value0)+'*'
            self.formula = formula[:-1]
            
        if self.key2 == 'divide':
            formula = '='
            for s in self.value0[self.key2]:
                testas = Formula(s)
                formula += str(testas.value0) +'/'
            self.formula = formula[:-1]
            
        if self.key2 == 'is_greater':
            formula = '=if('
            for s in self.value0[self.key2]:
                testas = Formula(s)
                formula += str(testas.value0) +'>'
            self.formula = formula[:-1]+";TRUE;FALSE)"
            
        if self.key2 == 'is_less':
            formula = '=if('
            for s in self.value0[self.key2]:
                testas = Formula(s)
                formula += str(testas.value0) +'<'
            self.formula = formula[:-1]+";TRUE;FALSE)"
            
        if self.key2 == 'is_equal':
            formula = '=if('
            for s in self.value0[self.key2]:
                testas = Formula(s)
                formula += str(testas.value0) +'='
            self.formula = formula[:-1]+";TRUE;FALSE)"
            
        if self.key2 == 'not':
            s = self.value0[self.key2]
            testas = Formula(s)
            self.formula = '=not('+testas.value0+')'
            
        if self.key2 == 'or':
            formula = '=or('
            for s in self.value0[self.key2]:
                __key2 = list(s.keys())[0]
                testas = Formula(s)
                formula += str(testas.value0) +';'
            self.formula = formula[:-1]+")"
        
        if self.key2 == 'and':
            formula = '=and('
            for s in self.value0[self.key2]:
                __key2 = list(s.keys())[0]
                testas = Formula(s)
                formula += str(testas.value0) +';'
            self.formula = formula[:-1]+")"
            
        if self.key2 == 'concat':
            formula = ''
            for s in self.value0[self.key2]:
                testas = Value(s)
                formula += str(testas.value)
            self.formula = formula
            
        if self.key2 == 'if':
            formula = '=if('
            condition = self.value0[self.key2][0]['is_greater']
            for c in condition:
                testas = Formula(c)
                formula += testas.value0 + '>'

            formula =  formula[:-1]+','                
            a = self.value0[self.key2][1]
            v = Formula(a)
            formula = formula + v.value0
            a = self.value0[self.key2][2]
            v = Formula(a)
            self.formula = formula + ","+ v.value0+")"

In [ ]:
import requests
import gspread
from time import sleep
import json


url = "https://www.wix.com/_serverless/hiring-task-spreadsheet-evaluator/jobs"
email = "viktoraspr@yahoo.com"

def read_data(url):
    resp = requests.get(url).json()
    submissionUrl = resp['submissionUrl']
    jobs = resp['jobs']
    return submissionUrl, jobs

def konvertuoti_to_result(listas):
    atsakymas = []
    for r in listas: 
        try:
            atsakymas.append({'number': int(r)})
            print
        except:
            if r == 'FALSE':
                atsakymas.append({'boolean': False})
            elif r == 'TRUE': 
                atsakymas.append({'boolean': True})
            else:
                atsakymas.append({'text': r})
                
    return atsakymas

def connect_worksheet():
    gc = gspread.service_account(filename='creds.json')
    sh = gc.open("wix_project")
    return sh

def write_to_spreadsheet(spread_data, sh):    
    sh.add_worksheet(title="Test", rows="100", cols="20")
    sheet = sh.worksheet("Test")
    
    for sp in spread_data:
        key = list(sp.keys())[0]
        y, x = key
        val = sp[key]
        sheet.update_cell(x, y, val)
        
    row = sheet.row_values(x)
    
    sh.del_worksheet(sheet)
        
    return row

def konvertuoti_to_jason(listas):
    atsakymas = []
    for r in listas: 
        try:
            atsakymas.append({'Number': int(r)})
            print
        except:
            if r == 'FALSE':
                atsakymas.append({'Boolean': False})
            elif r == 'TRUE': 
                atsakymas.append({'Boolean': True})
            else:
                atsakymas.append({'Text': r})
            
    return atsakymas

def form_prepare(data, row, sh):
    spread_data = []
    column = 1
    for d in data:
        key = list(d.keys())[0]
        if key == 'value':
            value = Value(d).value
        elif key == 'formula':
            value = Formula(d).formula
        spread_data.append({(column, row):value})
        column += 1
    
    print(spread_data)
    result = write_to_spreadsheet(spread_data, sh)
    sleep(10)
    return result

submissionUrl, jobs = read_data(url)

results = []
sh = connect_worksheet()

for datas in job:
    id_number = datas["id"]
    data = datas['data']
    if data == []:
        results.append({"id": id_number, "data": []})
        continue
    
    result = [] 
    for (i, d) in enumerate(data):
        r = i + 1 
        res = form_prepare(d, r, sh)
        result.append(res)


    results.append({"id": id_number, "data": result})